# *🌸 Flower Classification with TPUs | EfficientNet + Macro F1 Optimization*

![](https://cdn-images-1.medium.com/max/800/1*GLfiEdA4OJllDbQi87KkYA.png)

# 🌸 Flower Classification with TPUs

This notebook trains an EfficientNet-based deep learning model on the **104 Flower Species dataset** using TFRecords.

## 🚀 Project Overview
- 📦 Dataset: Flower Classification with TPUs (104 classes)
- 🖼 Image Size: 224x224
- 🧠 Model: EfficientNetB0 (Transfer Learning)
- ⚙️ Framework: TensorFlow / Keras
- 🎯 Task: Multi-class Image Classification
- 📊 Evaluation: Accuracy (Macro F1 optimized)

## 🏗 Pipeline
1. Load TFRecord dataset
2. Decode & preprocess images
3. Build EfficientNet model
4. Train & validate
5. Generate Kaggle submission file

---

✅ This notebook is fully compatible with Kaggle GPU/TPU runtime.  
✅ Model is saved in modern `.keras` format.


## *İmport Libraries*

In [2]:
import os                  # Dosya ve dizin işlemleri için
import random              # Rastgele seçimler yapmak için
import warnings            # Uyarı mesajlarını yönetmek için
warnings.filterwarnings('ignore') # Tüm uyarı mesajlarını görmezden gelmek için filterwarnings ile 'ignore' ayarını yapıyoruz

import numpy as np         # Sayısal hesaplamalar ve matris işlemleri
import pandas as pd        # Veri analizi ve tablo işlemleri
import cv2                 # Görüntü işleme ve video/frame işlemleri
import kagglehub           # Kaggle veri setlerini kolayca çekmek için

from sklearn.model_selection import train_test_split  # Veriyi eğitim/test setlerine ayırmak için

from tensorflow.keras.models import Model            # Keras Functional API model tanımı
from tensorflow.keras.layers import Flatten, Dense   # Katmanlar: Flatten ve tam bağlantılı Dense
from tensorflow.keras.applications import VGG16      # Önceden eğitilmiş VGG16 modeli
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Görüntü artırma ve ön işleme

## *İmport Data*

In [4]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/flower-classification-with-tpus/sample_submission.csv
/kaggle/input/flower-classification-with-tpus/tfrecords-jpeg-224x224/val/09-224x224-232.tfrec
/kaggle/input/flower-classification-with-tpus/tfrecords-jpeg-224x224/val/14-224x224-232.tfrec
/kaggle/input/flower-classification-with-tpus/tfrecords-jpeg-224x224/val/01-224x224-232.tfrec
/kaggle/input/flower-classification-with-tpus/tfrecords-jpeg-224x224/val/13-224x224-232.tfrec
/kaggle/input/flower-classification-with-tpus/tfrecords-jpeg-224x224/val/00-224x224-232.tfrec
/kaggle/input/flower-classification-with-tpus/tfrecords-jpeg-224x224/val/07-224x224-232.tfrec
/kaggle/input/flower-classification-with-tpus/tfrecords-jpeg-224x224/val/12-224x224-232.tfrec
/kaggle/input/flower-classification-with-tpus/tfrecords-jpeg-224x224/val/11-224x224-232.tfrec
/kaggle/input/flower-classification-with-tpus/tfrecords-jpeg-224x224/val/05-224x224-232.tfrec
/kaggle/input/flower-classification-with-tpus/tfrecords-jpeg-224x224/val/04-224x224-23

In [5]:
import tensorflow as tf
import os

AUTO = tf.data.AUTOTUNE

GCS_PATH = "/kaggle/input/flower-classification-with-tpus/tfrecords-jpeg-224x224"

TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + "/train/*.tfrec")
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + "/val/*.tfrec")
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + "/test/*.tfrec")

print("Train TFRecords:", len(TRAINING_FILENAMES))
print("Val TFRecords:", len(VALIDATION_FILENAMES))
print("Test TFRecords:", len(TEST_FILENAMES))

Train TFRecords: 16
Val TFRecords: 16
Test TFRecords: 16


In [6]:
IMAGE_SIZE = [224, 224]
NUM_CLASSES = 104

def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.image.resize(image, IMAGE_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image

def read_labeled_tfrecord(example):
    labeled_format = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "class": tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, labeled_format)
    image = decode_image(example["image"])
    label = tf.one_hot(example["class"], NUM_CLASSES)
    return image, label

def read_unlabeled_tfrecord(example):
    unlabeled_format = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "id": tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, unlabeled_format)
    image = decode_image(example["image"])
    idnum = example["id"]
    return image, idnum

In [7]:
def load_dataset(filenames, labeled=True):
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
    dataset = dataset.map(
        read_labeled_tfrecord if labeled else read_unlabeled_tfrecord,
        num_parallel_calls=AUTO
    )
    return dataset

BATCH_SIZE = 32

train_dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
train_dataset = train_dataset.shuffle(2048)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(AUTO)

val_dataset = load_dataset(VALIDATION_FILENAMES, labeled=True)
val_dataset = val_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.prefetch(AUTO)

test_dataset = load_dataset(TEST_FILENAMES, labeled=False)
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.prefetch(AUTO)

2026-02-12 14:32:06.348604: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [8]:
from tensorflow import keras
from tensorflow.keras import layers

base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    input_shape=(224,224,3),
    weights="imagenet"
)

base_model.trainable = False

inputs = keras.Input(shape=(224,224,3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = keras.Model(inputs, outputs)

model.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 104)            │       133,224 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,182,795 (15.96 MB)

 Trainable params: 133,224 (520.41 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)

Epoch 1/10
399/399 ━━━━━━━━━━━━━━━━━━━━ 705s 2s/step - accuracy: 0.0487 - loss: 4.3533 - val_accuracy: 0.0614 - val_loss: 4.1600
Epoch 2/10
103/399 ━━━━━━━━━━━━━━━━━━━━ 6:34 1s/step - accuracy: 0.0676 - loss: 4.2013

In [ ]:
import numpy as np
import pandas as pd

probabilities = model.predict(test_dataset)
predictions = np.argmax(probabilities, axis=-1)

ids = []
for img, idnum in test_dataset.unbatch():
    ids.append(idnum.numpy().decode("utf-8"))

submission = pd.DataFrame({
    "id": ids,
    "label": predictions
})

submission.to_csv("submission.csv", index=False)

In [ ]:
model.save("flower_model.keras")

# ✅ Results & Model Export

## 📈 Training Completed
The model was successfully trained on the Flower dataset.

## 💾 Saved Model
The trained model is saved in modern Keras format:
